In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os

savefolder = "/Users/your/savefolder"
datafolder = "/Users/your/datafolder"

## Figure 2 a-e Analysis of wall effect

In [ ]:
## Figure 2 a-e

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os


### stimuli in the whole rig
# exp_name = "zebra_sine"
# color = "blue"
# data = pd.read_pickle(r"%s/df_zebrafish_sinegrating.pkl" % datafolder)

exp_name = "medaka_sine"
color = "orange"
data = pd.read_pickle(r"%s/df_Medaka_sinegrating.pkl" % datafolder)

### stimuli in the restricted area
exp_name = "zebra_sine_restrict"
color = "blue"
data = pd.read_pickle(r"%s/df_zebrafish_sinegrating_moving_black.pkl" % datafolder)

# exp_name = "medaka_sine_restrict"
# color = "orange"
# data = pd.read_pickle(r"%s/df_Medaka_sinegrating_moving_white.pkl" % datafolder)

data.tail()

In [ ]:
## analyze experimental results in response to leftward and rightward stimuli
## Analyze the turn angle depending on the distance to the wall

stimulus = 4
fish_n = 30

## for figure in paper

data_frame = []
l_stim_data = data.loc[(data['stimulus_index'] == 1)]
r_stim_data = data.loc[(data['stimulus_index'] == 3)]
df = r_stim_data.copy()
df['bout_angle'] = r_stim_data['bout_angle'] * (-1)
    
merge_data = [l_stim_data, df]
big_data = pd.concat(merge_data)

trial_num = 15
time_start = 5
time_stop = 25
start_timing = 0
middle_timing = time_start
stop_timing = time_stop
dt = 1./ 90
binning1 = np.arange(start_timing, stop_timing-0.1, 0.05)
analysis_bin = np.arange(start_timing, stop_timing-0.1, 0.2)

columns = ["bin_n", "density", "pattern", "tbin"]
data_frame = [] #pd.DataFrame(columns=columns)


distance_to_wall = np.arange(0, 70, 10)
for d in range(len(distance_to_wall)-1):
    selected_data = big_data.loc[(big_data['center_distance']< (60 - distance_to_wall[d])) & (big_data['center_distance'] > (60 - distance_to_wall[d+1]))]
    
    fig, axs = plt.subplots(5, figsize=(4,9))
    
    stimuli = []

    for t in binning1:
        if t < 5:
            stimuli.append(0)
        else:
            stimuli.append(1)
    
    axs[0].axvline(x=5, color='black', alpha=0.5, linestyle="--")
    axs[0].plot(binning1, stimuli, color="red")
    axs[0].set_ylim(-1.1, 1.1)
    axs[0].axhline(y=0, color="black", linestyle="--")
    axs[0].set_ylabel("stimulus")
    axs[0].set_xlim(3, 15)
    
    print(len(selected_data["start_time_index"])/5000)
    axs[1].axvline(x=5, color='black', alpha=0.5, linestyle="--")
    axs[1].scatter(selected_data["start_time_index"], y=selected_data["bout_angle"], color=color, alpha=0.6, marker=".", s=0.5/(len(selected_data["start_time_index"])/9000), )
    axs[1].set_ylim(-100, 100)
    axs[1].set_xlim(3, 15) #rest + pulse)
    axs[1].axhline(y=0, color="black", linestyle="--")
    
    plt.tight_layout()
    plt.savefig("%s/fig_%s_distance%s_scatter.png" % (savefolder, exp_name, d),
                        transparent=True)
    plt.show()    

    
    fig, axs = plt.subplots(5, figsize=(4,9))
    axs[0].axvline(x=5, color='black', alpha=0.5, linestyle="--")
    axs[0].plot(binning1, stimuli, color="red")
    axs[0].set_ylim(-1.1, 1.1)
    axs[0].axhline(y=0, color="black", linestyle="--")
    axs[0].set_ylabel("stimulus")
    axs[0].set_xlim(3, 15)

    show_t = []

    ave_angles = []
    std_angles = []
    for t_bin in range(len(analysis_bin)-1):
        data_sub = selected_data.loc[(selected_data["start_time_index"] > analysis_bin[t_bin]) & (selected_data["start_time_index"] < analysis_bin[t_bin+1])]
        show_t.append((analysis_bin[t_bin+1] + analysis_bin[t_bin])/2)
        ave_angles.append(data_sub["bout_angle"].mean())
        std_angles.append(data_sub["bout_angle"].std()/np.sqrt(fish_n))

    axs[2].axvline(x=5, color='black', alpha=0.5, linestyle="--")

    axs[2].fill_between(show_t, np.array(ave_angles) - np.array(std_angles), np.array(ave_angles) + np.array(std_angles), color=color, alpha=0.3)
    axs[2].plot(show_t, ave_angles, color=color, alpha=1, )
    axs[2].set_ylim(-50, 50)
    axs[2].axhline(y=0, color="black", linestyle="--")
    axs[2].set_xlim(3, 15)
    
    
    plt.tight_layout()
    plt.savefig("%s/fig_%s_distance%s_average.svg" % (savefolder, exp_name, d),
                        transparent=True)
    plt.show() 

In [ ]:
distance_to_wall = np.arange(0, 70, 10)
ave_angles_whole, std_angles_whole = [], []
for d in range(len(distance_to_wall)-1):
    selected_data = big_data.loc[(big_data['center_distance']< (60 - distance_to_wall[d])) & (big_data['center_distance'] > (60 - distance_to_wall[d+1]))]
    data_sub = selected_data.loc[(selected_data["start_time_index"] > 10) & (selected_data["start_time_index"] < 25)]
    ave_angles_whole.append(data_sub["bout_angle"].mean())
    std_angles_whole.append(data_sub["bout_angle"].std()/np.sqrt(fish_n))                


In [ ]:
showx = distance_to_wall[:-1]
plt.plot(showx, ave_angles_whole, color=color, linestyle="--", marker="o")
plt.fill_between(showx, np.array(ave_angles_whole) - np.array(std_angles_whole), np.array(ave_angles_whole) + np.array(std_angles_whole), color=color, alpha=0.3)
plt.ylim(-15, 45)
plt.axhline(0, color="black", linestyle="--")
plt.savefig("%s/medaka_whole_res_dis_resrigonly.svg" % (savefolder))
# plt.ylim(0, 40)

In [ ]:
## compare how much the turning is suppressed by the wall
## 
showx_zs = [0, 10, 20,30, 40,]
showx_ms = [3, 13, 23,33, 43,]
fish_number = 31

      
l_stim_data = data.loc[(data['stimulus_index'] == 1)]
r_stim_data = data.loc[(data['stimulus_index'] == 3)]
l2_stim_data = data.loc[(data['stimulus_index'] == 5)]
r2_stim_data = data.loc[(data['stimulus_index'] == 7)]
df = r_stim_data.copy()
df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

df2 = r2_stim_data.copy()
df2['bout_angle'] = r2_stim_data['bout_angle'] * (-1)
    
merge_data = [l_stim_data, df, l2_stim_data, df2]
big_data = pd.concat(merge_data)
        
distance_to_wall = np.arange(0, 60, 10)
ave_angles_whole, std_angles_whole = [], []
deviation_ave, deviation_ste = [], []

for d in range(len(distance_to_wall)-1):
#     print(d)
    deviation_d = []

    for fish in range(fish_number):

        selected_data = big_data.loc[(big_data['fish_index'] == fish) & (big_data['center_distance']< 10) & (big_data['center_distance'] > 0)]
        data_sub = selected_data.loc[(selected_data["start_time_index"] > 10) & (selected_data["start_time_index"] < 25)]
        center_ang = data_sub["bout_angle"].mean()
        selected_data = big_data.loc[(big_data['fish_index'] == fish) & (big_data['center_distance']< (60 - distance_to_wall[d])) & (big_data['center_distance'] > (60 - distance_to_wall[d+1]))]
        data_sub = selected_data.loc[(selected_data["start_time_index"] > 10) & (selected_data["start_time_index"] < 25)]
        deviation_d.append(data_sub["bout_angle"].mean() - center_ang)

    deviation_ave.append(np.nanmean(deviation_d))
    deviation_ste.append(np.nanstd(deviation_d)/np.sqrt(fish_number))

print(deviation_ave)
print(deviation_ste)


In [ ]:
z_whole_dev_ave = [-25.33239944217422, -23.852899893084665, -18.040117067402313, -12.867724877950236, -6.297446879758722]
z_whole_dev_ste = [2.0248607098462648, 1.7801381344001175, 1.9312973368514312, 1.1191374963583096, 0.6273150125466831]
m_whole_dev_ave =[-38.80999483483591, -38.061034562023146, -38.100628927635622, -29.08004669255537, -15.507523462354678]
m_whole_dev_ste =[3.426523755644862, 3.241422344951606, 2.6765856375018977, 2.2453299781327374, 1.6046445309778317]
z_restr_dev_ave = [0.4853243280364952, 3.7949832522260945, 4.384792205246315, 3.0417940581166545, 3.1998332607502147]
z_restr_dev_ste = [2.0494075213534386, 1.9154798966318793, 1.8742180355853217, 2.0401721990719204, 1.9068000252383521]
m_restr_dev_ave = [-19.615842214471282, -10.022295163234985937, -3.09431062848186698, -1.9261421037229394, -1.3671018989670916]
m_restr_dev_ste = [5.617899014040034, 3.238062957696074, 2.7689549096626127, 2.5354430287452803, 2.383200383871117]

In [ ]:
showx_zs = [0, 10, 20,30, 40,]
showx_ms = [3, 13, 23,33, 43,]

fig, axs = plt.subplots(1, figsize=(2,1))
plt.bar(showx_zs, z_whole_dev_ave, color="blue", width=2)
plt.errorbar(showx_zs, z_whole_dev_ave, yerr=z_whole_dev_ste, color="blue", fmt=" ")
plt.bar(showx_ms, m_whole_dev_ave, width=2,color="orange")
plt.errorbar(showx_ms, m_whole_dev_ave, yerr=m_whole_dev_ste, color="orange", fmt=" ")
plt.ylim(-45, 7)
plt.axhline(0, color="black",linestyle="--")
plt.tight_layout()
plt.savefig("%s/fig_sine_whole_deviation.svg" % (savefolder),
                        transparent=True)

In [ ]:
## ## for statistics
# Make a DataFrame with columns:
# 'fish_id', 'species', 'spatial_bin', 'turning_angle', "whole_or_not" (whole is 0, restrict is 1)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_stats = pd.DataFrame(columns=['fish_id', 'species', 'spatial_bin', 'turning_angle', "whole_or_not"])

species = ["zebrafish", "medaka", "zebrafish", "medaka"]
exp_names = ["zebra_sine", "medaka_sine", "zebra_sine_restrict", "medaka_sine_restrict"]
data_names = ["zebrafish_sinegrating", "Medaka_sinegrating",
             "zebrafish_sinegrating_moving_black", "Medaka_sinegrating_moving_white"]
whole_or_nots =[0, 0, 1, 1]
fish_number_id = 0

for exp_name_i, exp_name in enumerate(exp_names):
    
    data = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, data_names[exp_name_i]))
    
    data_frame = []
    if max(data["stimulus_index"]) > 4:
        
        l_stim_data = data.loc[(data['stimulus_index'] == 1)]
        r_stim_data = data.loc[(data['stimulus_index'] == 3)]
        l2_stim_data = data.loc[(data['stimulus_index'] == 5)]
        r2_stim_data = data.loc[(data['stimulus_index'] == 7)]
        df = r_stim_data.copy()
        df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

        df2 = r2_stim_data.copy()
        df2['bout_angle'] = r2_stim_data['bout_angle'] * (-1)
        merge_data = [l_stim_data, df, l2_stim_data, df2]
    else:
        data_frame = []
        l_stim_data = data.loc[(data['stimulus_index'] == 1)]
        r_stim_data = data.loc[(data['stimulus_index'] == 3)]
        df = r_stim_data.copy()
        df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

        merge_data = [l_stim_data, df]
        
    big_data = pd.concat(merge_data)

    trial_num = 15
    time_start = 5
    time_stop = 25
    start_timing = 0
    middle_timing = time_start
    stop_timing = time_stop
    dt = 1./ 90
    binning1 = np.arange(start_timing, stop_timing-0.1, 0.05)
    analysis_bin = np.arange(start_timing, stop_timing-0.1, 0.2)

    columns = ["bin_n", "density", "pattern", "tbin"]
    data_frame = [] #pd.DataFrame(columns=columns)

    distance_to_wall = np.arange(0, 70, 10)
    fish_number = max(big_data["fish_index"])
    
    for fish in range(fish_number):
        fish_number_id = fish_number_id+1
        fish_big_data = big_data.loc[(big_data["fish_index"]== fish)]
            
        for d in range(len(distance_to_wall)-1):
            selected_data = fish_big_data.loc[(fish_big_data['center_distance']< (60 - distance_to_wall[d])) & (fish_big_data['center_distance'] > (60 - distance_to_wall[d+1]))]
            ave_angle = np.nanmean(selected_data["bout_angle"])
            
            df2 = pd.DataFrame({
            'fish_id': fish_number_id,
                    'species': species[exp_name_i],
                    'spatial_bin': d,
            "turning_angle": ave_angle,
                "whole_or_not": whole_or_nots[exp_name_i]
                }, index=[0]) 
            df_stats = pd.concat([df_stats, pd.DataFrame(df2)], ignore_index=True)
            
df_stats.tail()

In [ ]:
## want to do the statistics
## compare between zebrafish and medaka across each
## this is from Claud

# !pip install pingouin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
import pingouin as pg

# Assuming your data is in a DataFrame with columns:
# 'fish_id', 'species', 'spatial_bin', 'turning_angle'

# Perform two-way mixed ANOVA
df = df_stats.loc[df_stats["whole_or_not"]==1]
aov = pg.mixed_anova(data=df, dv='turning_angle', 
                     within='spatial_bin', between='species', 
                     subject='fish_id')

# Print ANOVA results
print(aov)

# If interaction is significant, perform simple main effects analysis
# Simple effect of species at each spatial bin
simple_effects = []
for bin_n in range(0, 6):

    bin_data = df[df['spatial_bin'] == bin_n]
    result = pg.ttest(bin_data[bin_data['species'] == 'zebrafish']['turning_angle'],
                          bin_data[bin_data['species'] == 'medaka']['turning_angle'],
                          paired=False)

    result['spatial_bin'] = bin_n
    simple_effects.append(result)
    
simple_effects_df = pd.concat(simple_effects).reset_index(drop=True)

    
    # Apply correction for multiple comparisons
from statsmodels.stats.multitest import multipletests
_, simple_effects_df['p_adj'], _, _ = multipletests(
        simple_effects_df['p-val'], method='bonferroni')
    
print(simple_effects_df)


# If main effect of spatial bin is significant, perform pairwise comparisons
medaka_df = df.loc[df["species"]=='zebrafish']
zebrafish_df = df.loc[df["species"]=='medaka']
print("medaka!")
posthoc_m = pg.pairwise_ttests(data=medaka_df, dv='turning_angle', 
                                 within='spatial_bin', subject='fish_id',
                                 padjust='bonferroni')
print(posthoc_m)

print("zebrafish!")
posthoc_z = pg.pairwise_ttests(data=zebrafish_df, dv='turning_angle', 
                                 within='spatial_bin', subject='fish_id',
                                 padjust='bonferroni')
print(posthoc_z)

   

In [ ]:
## compare between whole and restricted
print("compare between whole and restricted")
species_category = ["medaka", "zebrafish"]

for sp_n in species_category:
    df2 = df_stats.loc[df_stats["species"]== sp_n]
    print("result of ", sp_n)
    
    aov = pg.mixed_anova(data=df2, dv='turning_angle', 
                     within='spatial_bin', between='whole_or_not', 
                     subject='fish_id')

    # Print ANOVA results
    print(aov)
    
    simple_effects = []
    for bin_n in range(0, 6):

        bin_data = df2[df2['spatial_bin'] == bin_n]
        result = pg.ttest(bin_data[bin_data['whole_or_not'] == 0]['turning_angle'],
                              bin_data[bin_data['whole_or_not'] == 1]['turning_angle'],
                              paired=False)

        result['spatial_bin'] = bin_n
        simple_effects.append(result)

    simple_effects_df = pd.concat(simple_effects).reset_index(drop=True)


        # Apply correction for multiple comparisons
    from statsmodels.stats.multitest import multipletests
    _, simple_effects_df['p_adj'], _, _ = multipletests(
            simple_effects_df['p-val'], method='bonferroni')

    print(simple_effects_df)


## Figure 2 f-j Stimuli with different size of motion disk

In [ ]:
## bring the dataset from the actual turning at the sandblasted rig
## for each fish, see the height, amplitude and calculate the saturation point?
## for each fish, fit with the curve
## measure the threshold size


import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy import interpolate

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os

scales = [0.1, 0.2,0.26,0.33, 0.5, 1]
directions = [-90, 90]

### dots
## datasets are already screened based on the activity of fish.
## only the fish who swam close to the center, in response to the open-loop converging stimuli, were screened.
exp_name1 = "diff_dot_disk_size_exp_zebra_sandblast_good_rig_last" 
good_fishs1 =["2024_11_16_fish106", "2024_11_16_fish115", "2024_11_16_fish119", "2024_11_17_fish122", "2024_11_17_fish133"]

exp_name2 = "diff_dot_disk_size_exp_zebrafish_sandblast_good_rig_last_last"
good_fishs2 = ["2025_01_21_fish001", "2025_01_21_fish003", "2025_01_21_fish010", "2025_01_21_fish011", 
"2025_01_22_fish021", ]

exp_name3 = "diff_dot_disk_size_exp_medaka_sandblast_good_rig_last"
good_fishs3 = ["2024_11_16_fish129", "2024_11_17_fish114", "2024_11_17_fish116"]

exp_name4 = "diff_dot_disk_size_exp_medaka_sandblast_good_rig_last_last"
good_fishs4=["2025_01_13_fish001", "2025_01_13_fish003", "2025_01_13_fish007", "2025_01_13_fish009", "2025_01_13_fish010", "2025_01_13_fish011",
            "2025_01_14_fish016", "2025_01_14_fish025" 
]

color12 = "blue"
color34 = "orange"

data1 = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, exp_name1))
data2 = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, exp_name2))
data3 = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, exp_name3))
data4 = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, exp_name4))


### make a summary graphs for different size of discs
## different size of disk
exp_names = ["diff_dot_disk_size_exp_medaka_sandblast_good_rig_last_2data", "diff_dot_disk_size_exp_zebra_sandblast_good_rig_last_2data",]

## different size of disk
angless = [19,  36, 50, 65, 80.5,83]
scales = [0.1*30, 0.2*30,0.26*30,0.33*30, 0.5*30, 1*30]

colors = [ "orange" ,"blue",]


stimulus_pattern = []
patterns = [0, 2,4,6,8, 10]

trial_num = 20
time_start = 5
time_stop = 25
start_timing = 0
middle_timing = time_start
stop_timing = time_stop
dt = 1./ 90
binning1 = np.arange(start_timing, stop_timing-0.1, 0.05)
analysis_bin = np.arange(start_timing, stop_timing-0.1, 0.2)


# Parameters
r = 2  # radius of circular dot (mm)
d = 5  # speed of dot movement along x-axis (mm/s)

def interpolate_data(time, values, new_time_points, method='linear'):
        
    # Create interpolation function
    f = interpolate.interp1d(time, values, kind=method, bounds_error=False, fill_value="extrapolate")
    # Get new values using the interpolation function
    new_values = f(new_time_points)
    return new_values

# Functions for calculating angles and derivatives
def calculate_theta(x, y, h):
    distance = np.sqrt(x**2 + y**2 + h**2)
    theta = np.arccos(h / distance) * 180 / np.pi
    return theta

def calculate_phi(x, y, h, r):
    distance = np.sqrt(x**2 + y**2 + h**2)
    phi = np.arcsin(r / distance) * 180 / np.pi
    return phi

def dtheta_dx(x, y, h):
    distance = np.sqrt(x**2 + y**2 + h**2)
    
    # Use numpy's where to handle array inputs
    denominator = distance**2 * np.sqrt(distance**2 - h**2)
    # Create a condition mask
    mask = (distance**2 <= h**2)
    
    # Use where to handle the condition
    result = np.where(mask, 0, (x * h) / denominator * 180 / np.pi)
    return result

def dphi_dx(x, y, h, r):
    distance = np.sqrt(x**2 + y**2 + h**2)
    
    # Create a condition mask
    mask = (distance <= r)
    
    # Handle array inputs
    denominator = distance**2 * np.sqrt(1 - (r/distance)**2)
    result = np.where(mask, 0, -r * x / denominator * 180 / np.pi)
    return result

def dtheta_dt(x, y, h, speed):
    return dtheta_dx(x, y, h) * speed

def dphi_dt(x, y, h, r, speed):
    return dphi_dx(x, y, h, r) * speed

# Motion energy definitions
def energy_abs_product(x, y, h, r, speed):
    """Absolute product of angular rates"""
    dt_dt = dtheta_dt(x, y, h, speed)
    dp_dt = dphi_dt(x, y, h, r, speed)
    return np.abs(dt_dt * dp_dt)


# Function to calculate total energy in a disk
def calculate_total_energy(disk_radius, energy_func, h, r, speed, num_samples=500):
    """
    Calculate total energy in a disk by numerical integration.
    """
    # Create a grid of points within the disk
    theta = np.linspace(0, 2*np.pi, num_samples)
    # Start with a small positive value to avoid the origin
    radius = np.linspace(0.5, disk_radius, num_samples//10)  # Start at 0.5 instead of 0
    
    # Convert to Cartesian coordinates
    theta_grid, radius_grid = np.meshgrid(theta, radius)
    x_grid = radius_grid * np.cos(theta_grid)
    y_grid = radius_grid * np.sin(theta_grid)
    
    # Calculate energy at each point
    energy_values = energy_func(x_grid, y_grid, h, r, speed)
    
    # Integrate over the disk (using polar coordinates)
    # In polar coordinates, dA = r dr dθ
    total_energy = 0
    for i in range(len(radius)-1):
        r_inner = radius[i]
        r_outer = radius[i+1]
        r_mid = (r_inner + r_outer) / 2
        for j in range(len(theta)-1):
            theta_span = theta[j+1] - theta[j]
            area_element = r_mid * (r_outer - r_inner) * theta_span
            total_energy += energy_values[i, j] * area_element
            
    return total_energy


# Calculate total energy for different disk radii

def calculate_total_energy_fit(disk_radii, *p):
    h, Amp = p
    total_energy_abs_product = []
        
    for disk_radius in disk_radii:
        total_energy_abs_product.append(Amp * calculate_total_energy(disk_radius, energy_abs_product, h, r, d))
        
    return total_energy_abs_product

fig, axs = plt.subplots(2, 2, figsize=(5, 5))

threshold_means, threshold_stds, threshold_Data = [], [], []

for exp_name_i, exp_name in reversed(list(enumerate(exp_names))):
    print(exp_name)
    
    if exp_name_i == 0:
        # medaka
        Data1, Data2 = data3, data4
        color = color34
        good_Fishs1, good_Fishs2 = good_fishs3, good_fishs4
        
    else:
        Data1, Data2 = data1, data2
        color = color12
        good_Fishs1, good_Fishs2 = good_fishs1, good_fishs2
        
    good_fishs_n = len(good_Fishs1) + len(good_Fishs2)
    good_fishs_x = good_Fishs1 + good_Fishs2
     
    selected_fish_data2 = pd.concat([Data1.loc[Data1['fishName'].isin(good_Fishs1)], Data2.loc[Data2['fishName'].isin(good_Fishs2)]], ignore_index=True)

    threshold_sizes = []
    for fish in good_fishs_x:
        selected_fish_data = selected_fish_data2.loc[(selected_fish_data2['fishName'] == fish)]
        
        ave_angles_patterns, ste_angles_patterns = [], []
        showx = scales #angless

        for pattern in range(6):

            stimulus_pattern_n = pattern
            r_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == 2 * pattern+1)]

            stimulus_pattern_n = pattern + len(patterns)
            l_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == 2 * pattern)] 

            df = r_stim_data.copy()
            df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

            merge_data = [l_stim_data, df]
            big_data = pd.concat(merge_data)
            big_data_center = big_data.loc[(big_data['center_distance'] < 30)]

            big_data_after5 = big_data_center.loc[(big_data_center['start_time_index'] > 6) & (big_data_center['start_time_index'] < 24)]
            ave_angles, std_angles = [], []

            ave_angle = big_data_after5["bout_angle"].mean()
            ave_angles_patterns.append(ave_angle)

        scale_0_fit = np.linspace(0, 30, 50)

        showx_fit = [0] + showx 
        ave_angles_patterns_fit =  [0] + ave_angles_patterns

        axs[0, exp_name_i].plot(showx_fit, ave_angles_patterns_fit, color=color, linestyle=" ", marker="o", alpha=0.8)


        N_early_points = 2 

        if exp_name_i == 1:
            showx_fit = [0] + showx[:N_early_points]
            ave_angles_patterns_fit =  [0] + ave_angles_patterns[:N_early_points]
        else:
            N_early_points = 4
            showx_fit = [0] + showx[:N_early_points] 
            ave_angles_patterns_fit =  [0] + ave_angles_patterns[:N_early_points] 

        new_showx_fit = np.linspace(min(showx_fit), max(showx_fit), 10) 
        new_ave_angles_patterns_fit = interpolate_data(showx_fit, ave_angles_patterns_fit, new_showx_fit, method='linear')

        ## here the fitting will show
        p0=[5, 1/3]
        coeff_u, var_matrix = curve_fit(calculate_total_energy_fit, new_showx_fit, new_ave_angles_patterns_fit,
                                       p0=p0, maxfev=5000, bounds = ((1, 0),(7, 1)))
        out_fit = calculate_total_energy_fit(scale_0_fit, *coeff_u)
        height, Amp = coeff_u
        print("height:",height,", Amp:",Amp)

        # For the actual data
        # Use a smooth interpolation for the actual data
        actual_interp = interpolate.make_interp_spline(([0] + showx), ([0] + ave_angles_patterns), k=1)
        actual_curve = actual_interp(scale_0_fit)
        # Calculate the difference between theoretical and actual curves
        difference = out_fit - actual_curve

        # Method 3: Find where actual curve reaches X% of its maximum value
        def find_saturation_by_max_percent(percent=0.95):
            max_value = np.max(actual_curve)
            threshold = percent * max_value
            indices = np.where(out_fit >= threshold)[0]
            if len(indices) > 0:
                return scale_0_fit[indices[0]]
            else:
                print("None!")
                return None

        saturation_max = find_saturation_by_max_percent(percent=0.95)
        threshold_sizes.append(saturation_max)

        axs[0, exp_name_i].plot(scale_0_fit, out_fit, color=colors[exp_name_i], alpha=0.2)#label='Theoretical motion energy curve')
        axs[0, exp_name_i].plot(scale_0_fit, actual_curve, color="black", alpha=0.2)#label='Theoretical motion energy curve')
        axs[0, exp_name_i].axvline(x=saturation_max, color=colors[exp_name_i], linestyle='--', alpha=0.2)
        axs[0, exp_name_i].set_ylim(-5, 60)
    threshold_means.append(np.average(threshold_sizes))
    threshold_stds.append(np.std(threshold_sizes))
    threshold_Data.append(threshold_sizes)


plt.tight_layout()
plt.savefig('%s/total_motion_energy_fitting_threshold_each_fish_30_2.svg' % savefolder)
plt.show()


In [ ]:
threshold_Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Example data - replace with your actual data
# Group 1: 10 datapoints
group1 = threshold_Data[0]

# Group 2: 13 datapoints
group2 = threshold_Data[1]

# Calculate averages for the bar heights
avg_group1 = np.mean(group1)
avg_group2 = np.mean(group2)
std_group1 = np.std(group1)
std_group2 = np.std(group2)
print("avg:",avg_group1, "std:",std_group1)
print("avg:",avg_group2, "std:",std_group2)

# Prepare data for seaborn in a pandas DataFrame
data = pd.DataFrame({
    'Group': ['Group 1']*len(group1) + ['Group 2']*len(group2),
    'Value': group1 + group2
})

# Create the figure and axes
fig, ax = plt.subplots(figsize=(1.8, 2))

# Create the bar plot first (so it's in the background)
bar_positions = [0, 1]  # Positions for the two groups
bar_width = 0.6
bar_colors = ['blue', 'orange']#['#AECEF4', '#FFCCA8']  # Light blue and light orange for bars

bars = ax.bar(bar_positions, [avg_group1, avg_group2], width=bar_width, 
              color=bar_colors, alpha=0.3,)# edgecolor='black', linewidth=1)

# Add a swarm plot on top
swarm = sns.swarmplot(x='Group', y='Value', data=data, ax=ax,
                      palette=['blue', 'orange'], size=8, alpha=0.8,)

# Add statistical annotations if needed
# For example, you could add a p-value or significance indicator
from scipy import stats
t_stat, p_value = stats.ttest_ind(group1, group2)
significance = "**" if p_value < 0.01 else ("*" if p_value < 0.05 else "ns")
print(p_value)

# Adjust layout and set a clean style
plt.tight_layout()
sns.despine(left=False, bottom=False)  # Remove the top and right spines
plt.savefig('%s/total_motion_energy_fitting_threshold_dot_30_2".svg' % savefolder)
plt.show()

t_stat, p_welch = stats.ttest_ind(group1, group2, equal_var=False)

# 2. Mann-Whitney U test (non-parametric)
u_stat, p_mannwhitney = stats.mannwhitneyu(group1, group2, alternative='two-sided')

# 3. Check normality of each group (Shapiro-Wilk test)
_, p_normal1 = stats.shapiro(group1)
_, p_normal2 = stats.shapiro(group2)
are_normal = (p_normal1 > 0.05) and (p_normal2 > 0.05)

# Choose the most appropriate test based on normality
if are_normal:
    test_name = "Welch's t-test"
    p_value = p_welch
    stat_value = t_stat
    stat_name = "t"
else:
    test_name = "Mann-Whitney U test"
    p_value = p_mannwhitney
    stat_value = u_stat
    stat_name = "U"

# Add significance symbols
significance = "**" if p_value < 0.01 else ("*" if p_value < 0.05 else "ns")

# Create the visualization (same as before with minor adjustments)
# ...rest of the plotting code as in my previous response...

# Update the statistical annotation
ax.text(0.5, max(max(group1), max(group2)) + 0.5, 
        f'{test_name}: {stat_name}={stat_value:.2f}, p={p_value:.4f} {significance}',
        ha='center', va='bottom', fontsize=11)

plt.show()

# Print detailed statistics for the methods section
print(f"\nStatistical summary for methods section:")
print(f"Normality tests: Group 1 p={p_normal1:.4f}, Group 2 p={p_normal2:.4f}")
print(f"Selected statistical test: {test_name}")
print(f"Statistical results: {stat_name}={stat_value:.3f}, p={p_value:.4f}{significance}")

"""Statistical Analysis

Statistical analyses were performed using Python (version X.X) with SciPy (version X.X) and NumPy (version X.X) libraries. Data are presented as mean ± standard error of the mean (SEM). Sample sizes were n=10 for group 1 and n=12 for group 2. 

The normality of data distribution in each group was assessed using the Shapiro-Wilk test. [CHOOSE ONE OF THE FOLLOWING BASED ON YOUR RESULTS:]

[IF DATA IS NORMAL:] As both groups showed normal distribution (Shapiro-Wilk test, p > 0.05), statistical comparisons between groups were performed using Welch's t-test, which does not assume equal variances between groups.

[IF DATA IS NOT NORMAL:] As the data did not meet the assumptions of normality (Shapiro-Wilk test, p < 0.05), statistical comparisons between groups were performed using the non-parametric Mann-Whitney U test.

Differences were considered statistically significant at p < 0.05 (*), p < 0.01 (**), and p < 0.001 (***). All statistical plots were generated using Matplotlib (version X.X) and Seaborn (version X.X)."""

## Figure 2 k-n Response to inconsistent stimuli

In [ ]:
## inside_outside
## only the fish who swam in the middle of the dish in response to open-loop converging stimuli were screened
# good_fishs_m_sand_good = ['2024_11_03_fish000', '2024_11_03_fish002', '2024_11_03_fish004', '2024_11_03_fish005', '2024_11_04_fish000', '2024_11_04_fish001', '2024_11_04_fish002', '2024_11_04_fish003', '2024_11_04_fish005', '2024_11_04_fish009', '2024_11_04_fish010', '2024_11_04_fish011', '2024_11_04_fish012', '2024_11_04_fish013', '2024_11_04_fish014', '2024_11_04_fish015', '2024_11_04_fish016', '2024_11_04_fish017', '2024_11_04_fish018', '2024_11_04_fish019', '2024_11_04_fish020', '2024_11_04_fish021', '2024_11_05_fish001', '2024_11_05_fish002', '2024_11_05_fish003', '2024_11_05_fish004', '2024_11_05_fish005', '2024_11_05_fish007', '2024_11_05_fish009', '2024_11_05_fish011', '2024_11_05_fish012', '2024_11_05_fish013']
good_fishs_m_sand_good =["2024_11_03_fish000",
"2024_11_03_fish002",
"2024_11_03_fish004",
"2024_11_04_fish005",
"2024_11_04_fish010",
"2024_11_04_fish012",
"2024_11_04_fish016",
"2024_11_04_fish020",
"2024_11_05_fish001",
"2024_11_05_fish004",
"2024_11_05_fish013",
]
good_fishs_z_sand_good = ["2024_11_03_fish011",
"2024_11_03_fish014",
"2024_11_03_fish020",
"2024_11_03_fish022",
"2024_11_03_fish025",
"2024_11_03_fish028",]

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats import norm

## inside_outside
exp_names = ["inside_outside_exp_medaka_sandblast_good_rig","inside_outside_exp_zebra_sandblast_good_rig"]


## different size of disk
angless = [[19, 27, 36, 48, 57, 65, 80.5,], [19, 27, 36, 48, 57, 65, 80.5,]]

colors = ["orange" ,"blue"]

good_fishss = [good_fishs_m_sand_good, good_fishs_z_sand_good]

datafolder = "/Users/yasukoisoe/PycharmProjects/fishfishfish/undulation/data_240908_spatial"
savefolder = "/Users/yasukoisoe/Dropbox/Yasuko/Project_Harvard/papers/paper_working_memory/data/"
stimulus_pattern = []
patterns = [0, 2,4,6,8, 10, 12]

trial_num = 20
time_start = 5
time_stop = 25
start_timing = 0
middle_timing = time_start
stop_timing = time_stop
dt = 1./ 90
binning1 = np.arange(start_timing, stop_timing-0.1, 0.05)
analysis_bin = np.arange(start_timing, stop_timing-0.1, 0.2)

plt.figure(figsize=(3, 2))

for exp_name_i, exp_name in enumerate(exp_names):

    data = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, exp_name))
    good_fishs = good_fishss[exp_name_i]
    selected_fish_data = data.loc[data['fishName'].isin(good_fishs)]

    ave_angles_patterns, ste_angles_patterns = [], []
    showx = angless[exp_name_i]

    for pattern in range(7):

        stimulus_pattern_n = pattern
        r_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == 2 * pattern)]

        stimulus_pattern_n = pattern + len(patterns)
        l_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == 2 * pattern + 1)] ## zebrafish
    #     l_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == pattern + 12)] ## medaka

        df = r_stim_data.copy()
        df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

        merge_data = [l_stim_data, df]
        big_data = pd.concat(merge_data)
        big_data_center = big_data.loc[(big_data['center_distance'] < 34)]

        big_data_after5 = big_data_center.loc[(big_data_center['start_time_index'] > 6) & (big_data_center['start_time_index'] < 15)]
        ave_angles = []
        std_angles = []
        for i in good_fishs:
            fish_data = big_data_after5.loc[big_data_after5["fishName"] == i]
            ave_angles.append(fish_data["bout_angle"].mean())

        ave_angles_patterns.append(np.nanmean(ave_angles))
        ste_angles_patterns.append(np.nanstd(ave_angles)/np.sqrt(len(good_fishs)))
        

    showx_show = [0] + showx
    ave_angles_patterns_show = [-ave_angles_patterns[-1]] + ave_angles_patterns
    ste_angles_patterns_show = [ste_angles_patterns[-1]] + ste_angles_patterns
    
    plt.fill_between(showx_show, np.array(ave_angles_patterns_show) - np.array(ste_angles_patterns_show), np.array(ave_angles_patterns_show) + np.array(ste_angles_patterns_show), 
                     color=colors[exp_name_i], alpha=0.3)
    plt.plot(showx_show, ave_angles_patterns_show, color=colors[exp_name_i], linestyle="--", marker="o")
#         plt.xlim(0, 5)
    plt.axhline(0, color="black", linestyle="--")

    plt.xlim(18, 50)
    plt.ylim(-50,50)

    plt.savefig("%s/sandblasted/inside_outside_exp_sandblast_good_rig2.svg" % (savefolder))


In [ ]:
## cancel size effect
from scipy.interpolate import interp1d

df_stats = pd.DataFrame(columns=['fish_id', 'species', 'cancel_size',])
species = ["medaka", "zebrafish"]

for exp_name_i, exp_name in enumerate(exp_names):

    data = pd.read_pickle(r"%s/df_%s.pkl" % (datafolder, exp_name))
    good_fishs = good_fishss[exp_name_i]
    
    cancel_sizes = []
    showx = angless[exp_name_i]
    for fish in good_fishs:
        
        each_selected_fish_data = data.loc[data['fishName']==fish]
        ave_each_fish = []
        
        for pattern in range(7):

    #         stimulus_pattern_n = pattern
            r_stim_data = each_selected_fish_data.loc[(each_selected_fish_data['stimulus_index'] == 2 * pattern)]

            stimulus_pattern_n = pattern + len(patterns)
            l_stim_data = each_selected_fish_data.loc[(each_selected_fish_data['stimulus_index'] == 2 * pattern + 1)] 

            df = r_stim_data.copy()
            df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

            merge_data = [l_stim_data, df]
            big_data = pd.concat(merge_data)
            big_data_center = big_data.loc[(big_data['center_distance'] < 34)]

            big_data_after5 = big_data_center.loc[(big_data_center['start_time_index'] > 6) & (big_data_center['start_time_index'] < 15)]
            ave_each_fish.append(big_data_after5["bout_angle"].mean())
            
        interpolation_function = interp1d(showx, ave_each_fish, kind='linear')
        # Generate a fine time grid for better zero-crossing detection
        fine_time_grid = np.linspace(showx[0], showx[-1], 1000)
        fine_data_points = interpolation_function(fine_time_grid)

        # Find where the interpolated data crosses zero
        zero_crossings = np.where(np.diff(np.sign(fine_data_points)))[0]
        zero_times = fine_time_grid[zero_crossings]
        cancel_sizes.append(float(zero_times))
        
        df2 = pd.DataFrame(
            {'fish_id':fish, 'species':species[exp_name_i], 'cancel_size':float(zero_times)},
            index=[0])
        df_stats = pd.concat([df_stats, pd.DataFrame(df2)], ignore_index=True)
        
    print(exp_name_i, "fish", cancel_sizes)
            
        
df_stats

In [ ]:
## make a bar plot
showx_zs = [1,1.2]
fig, axs = plt.subplots(1, figsize=(1.2,1.5))
medaka_cancels = [42.82432432432432, 41.83933933933934, 29.71171171171171, 33.4054054054054, 35.62162162162162, 39.192192192192195, 39.5, 26.756756756756758, 42.14714714714715, 44.97897897897898, 26.51051051051051]
zebra_cancels = [24.97147147147147, 22.816816816816818, 27.064564564564563, 29.65015015015015, 31.12762762762763, 26.2027027027027]

cancelsize_ave = [np.average(zebra_cancels), np.average(medaka_cancels)]
cancelsize_ste = [np.std(zebra_cancels)/np.sqrt(len(zebra_cancels)), np.std(medaka_cancels)/np.sqrt(len(medaka_cancels))]
colors = ["blue", "orange"]
plt.bar(showx_zs, cancelsize_ave, color=colors, width=0.15)
plt.errorbar(showx_zs, cancelsize_ave, yerr=cancelsize_ste, color="black", fmt=" ")

plt.tight_layout()
savefolder = "/Users/yasukoisoe/Dropbox/Yasuko/Project_Harvard/papers/paper_working_memory/data"
plt.savefig("%s/sandblasted/inside_outside_exp_sandblast_good_rig_cancelsize.svg" % (savefolder),
                        transparent=True)